In [ ]:
# -*- coding: utf-8 -*-
"""
Script de Exemplo para "Previsão" da Mega-Sena usando LSTM - Versão Aprimorada.

AVISO FUNDAMENTAL:
A Mega-Sena é um jogo de azar. Os sorteios são eventos aleatórios e independentes.
Não há NENHUMA EVIDÊNCIA CIENTÍFICA de que resultados passados possam prever
resultados futuros. Este script é APENAS UM EXERCÍCIO TÉCNICO de Machine Learning
e NÃO DEVE SER USADO como uma ferramenta para escolher números ou esperar ganhos.
As previsões, gráficos e métricas geradas refletem o desempenho do modelo nos dados
históricos e NÃO indicam capacidade real de previsão futura. USE POR SUA CONTA E RISCO,
ENTENDENDO QUE É UM EXPERIMENTO EDUCACIONAL.
"""

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split # Usaremos para validação, mas teste será cronológico
from sklearn.preprocessing import MultiLabelBinarizer
# Use TensorFlow/Keras para a Rede Neural LSTM
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import requests # Para baixar os dados (opcional)
from io import StringIO # Para ler os dados baixados
import os
import warnings
import matplotlib.pyplot as plt # Para gráficos

# Ignorar warnings de performance do TensorFlow (opcional)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore', category=UserWarning, module='tensorflow')
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

# --- Parâmetros Configuráveis ---
DATA_URL = "https://servicebus2.caixa.gov.br/portaldeloterias/api/megasena/" # Fonte potencial (verificar formato e disponibilidade)
DATA_FILE = None # Defina como None para tentar baixar, ou coloque o caminho do arquivo CSV 'resultados_mega_sena.csv'
EXPORT_FILE = 'historico_e_previsoes_megasena.xlsx' # Nome do arquivo Excel para exportação

SEQUENCE_LENGTH = 10  # Número de sorteios anteriores a serem usados para prever o próximo
NUM_FEATURES = 60     # Números possíveis na Mega-Sena (1 a 60)
LSTM_UNITS = 128      # Número de neurônios na camada LSTM
DROPOUT_RATE = 0.3    # Taxa de dropout para regularização
EPOCHS = 100          # Número máximo de épocas de treinamento
BATCH_SIZE = 32       # Tamanho do lote para treinamento
TEST_SIZE_RATIO = 0.15 # Proporção do dataset para teste (sorteios mais recentes)

# --- Funções (incluindo as de download, preprocessamento, criação de sequências - mantidas da versão anterior) ---

def download_and_prepare_data(url=None, file_path=None):
    """
    Baixa os dados da Mega-Sena de uma URL ou carrega de um arquivo CSV local.
    Retorna um DataFrame do Pandas com os resultados.
    Formato esperado: Colunas com os números sorteados (ex: 'Bola1', 'Bola2', ..., 'Bola6').
    """
    print("Carregando dados dos sorteios...")
    df = None

    if url:
        try:
            print(f"Tentando baixar dados de: {url}")
            response = requests.get(url, verify=False, timeout=30) # Adicionado timeout
            response.raise_for_status()
            data = response.json()

            if isinstance(data, list) and data and 'listaDezenas' in data[0]:
                 results = []
                 concursos = []
                 datas = []
                 for sorteio in data:
                     # Pega apenas as dezenas (e garante que são inteiros)
                     dezenas = sorted([int(d) for d in sorteio.get('listaDezenas', [])])
                     if len(dezenas) == 6: # Garante que temos 6 dezenas
                        results.append(dezenas)
                        concursos.append(sorteio.get('numero'))
                        datas.append(sorteio.get('dataApuracao'))

                 if not results:
                     print("Nenhum sorteio válido encontrado nos dados baixados.")
                     return None

                 df = pd.DataFrame(results, columns=[f'Bola{i+1}' for i in range(6)])
                 if concursos: df['Concurso'] = concursos
                 if datas: df['Data'] = pd.to_datetime(datas, dayfirst=True)

                 # Ordena pelo concurso (mais antigo primeiro) se a coluna existir
                 if 'Concurso' in df.columns:
                     df = df.sort_values(by='Concurso').reset_index(drop=True)
                 elif 'Data' in df.columns:
                     df = df.sort_values(by='Data').reset_index(drop=True)

                 print(f"Dados baixados e processados com sucesso ({len(df)} sorteios).")

            else:
                 print("Formato de dados JSON da API não reconhecido ou inesperado.")
                 # Tenta ler como CSV como fallback (menos provável para APIs modernas)
                 try:
                     print("Tentando ler a resposta como CSV...")
                     df = pd.read_csv(StringIO(response.text))
                     print("Dados lidos como CSV.")
                 except Exception as e_csv:
                     print(f"Não foi possível interpretar a resposta como JSON ou CSV: {e_csv}")
                     return None # Falha em ambas as tentativas

        except requests.exceptions.RequestException as e:
            print(f"Erro ao baixar dados: {e}")
            df = None # Garante que df seja None se o download falhar
        except Exception as e_proc:
             print(f"Erro ao processar os dados baixados: {e_proc}")
             return None

    # Se o download falhou ou não foi tentado, tenta o arquivo local
    if df is None and file_path and os.path.exists(file_path):
        print(f"Tentando carregar do arquivo local: {file_path}")
        try:
            df = pd.read_csv(file_path, sep=None, engine='python') # Tenta detectar separador
            # Verifica se a detecção funcionou (se temos mais de 1 coluna)
            if df.shape[1] < 6:
                 print(f"Arquivo CSV lido, mas parece ter poucas colunas ({df.shape[1]}). Verifique o separador.")
                 # Tenta alguns separadores comuns
                 for sep in [';', ',', '\t']:
                     try:
                         df_try = pd.read_csv(file_path, sep=sep)
                         if df_try.shape[1] >= 6:
                             df = df_try
                             print(f"Separador '{sep}' funcionou.")
                             break
                     except:
                         continue
            print(f"Dados carregados de {file_path}")
        except Exception as e_file:
            print(f"Erro ao carregar arquivo local: {e_file}")
            return None

    # Se ainda não temos DataFrame, sai
    if df is None:
        print("Nenhuma fonte de dados (URL ou arquivo) funcionou ou foi fornecida.")
        return None

    # --- Bloco de identificação e renomeação de colunas (aprimorado) ---
    bola_cols_found = []
    potential_col_names = [f'Bola{i+1}' for i in range(6)] + \
                          [f'bola{i+1}' for i in range(6)] + \
                          [f'Dezena{i+1}' for i in range(6)] + \
                          [f'dezena{i+1}' for i in range(6)] + \
                          [f'N{i+1}' for i in range(6)] # Adiciona mais padrões

    # Prioriza nomes exatos ou variações comuns
    for pattern_list in [[f'Bola{i+1}' for i in range(6)], [f'Dezena{i+1}' for i in range(6)]]:
        if all(col in df.columns for col in pattern_list):
            bola_cols_found = pattern_list
            break

    # Se não encontrou, tenta identificar heuristicamente
    if not bola_cols_found:
        numeric_cols = df.select_dtypes(include=np.number).columns
        potential_bola_cols = [c for c in numeric_cols if df[c].between(1, 60, inclusive='both').all() and df[c].notna().all()]
        if len(potential_bola_cols) >= 6:
            # Pega as primeiras 6 colunas que se encaixam no critério
            bola_cols_found = potential_bola_cols[:6]
            print(f"Colunas de bolas identificadas heuristicamente como: {bola_cols_found}")
        else:
            print(f"Erro: Não foi possível identificar 6 colunas com números entre 1 e 60.")
            print(f"Colunas encontradas: {list(df.columns)}")
            return None

    # Renomeia para o padrão 'BolaX' se necessário e seleciona
    rename_map = {found_col: f'Bola{i+1}' for i, found_col in enumerate(bola_cols_found)}
    df.rename(columns=rename_map, inplace=True)
    bola_cols = [f'Bola{i+1}' for i in range(6)]

    # Garante que as colunas das bolas são numéricas inteiras
    try:
        for col in bola_cols:
            df[col] = pd.to_numeric(df[col]).astype(int)
        print("Colunas das bolas verificadas e convertidas para inteiro.")
    except Exception as e_num:
        print(f"Erro ao converter colunas de bolas para numérico: {e_num}")
        return None

    # Seleciona e retorna apenas as colunas relevantes (Bolas e talvez Concurso/Data para referência)
    cols_to_keep = bola_cols
    if 'Concurso' in df.columns: cols_to_keep.append('Concurso')
    if 'Data' in df.columns: cols_to_keep.append('Data')

    final_df = df[cols_to_keep].copy()

    # Ordena novamente para garantir, caso a ordenação original tenha se perdido
    if 'Concurso' in final_df.columns:
        final_df = final_df.sort_values(by='Concurso').reset_index(drop=True)
    elif 'Data' in final_df.columns:
        final_df = final_df.sort_values(by='Data').reset_index(drop=True)

    print(f"Total de {len(final_df)} sorteios carregados e formatados.")
    return final_df # Retorna o DataFrame com Bolas e possivelmente Concurso/Data


def preprocess_data(df_balls_only):
    """
    Transforma os números sorteados (DataFrame apenas com colunas BolaX)
    em formato MultiLabelBinarizer (One-Hot Encoding para múltiplas labels).
    """
    print("Pré-processando os dados (codificação One-Hot)...")
    draws_list = df_balls_only.values.tolist()
    mlb = MultiLabelBinarizer(classes=list(range(1, NUM_FEATURES + 1)))
    encoded_data = mlb.fit_transform(draws_list)
    print(f"Dados transformados em formato binário ({encoded_data.shape[1]} features).")
    return encoded_data, mlb

def create_sequences(data, sequence_length):
    """
    Cria sequências de dados para o modelo LSTM.
    X: Sequências de 'sequence_length' sorteios.
    y: O sorteio seguinte a cada sequência.
    """
    print(f"Criando sequências de tamanho {sequence_length}...")
    X, y = [], []
    if len(data) <= sequence_length:
        print(f"Erro: Não há dados suficientes ({len(data)}) para criar sequências de tamanho {sequence_length}.")
        return np.array([]), np.array([])
    for i in range(len(data) - sequence_length):
        X.append(data[i:(i + sequence_length)])
        y.append(data[i + sequence_length])
    print(f"{len(X)} sequências criadas.")
    return np.array(X), np.array(y)

def build_lstm_model(sequence_length, num_features, lstm_units=128, dropout_rate=0.3):
    """ Constrói o modelo LSTM. """
    print("Construindo o modelo LSTM...")
    model = Sequential(name="Modelo_LSTM_MegaSena")
    model.add(Input(shape=(sequence_length, num_features)))
    model.add(LSTM(lstm_units, return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(lstm_units // 2))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_features, activation='sigmoid')) # Saída multi-label
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['binary_accuracy', tf.keras.metrics.AUC(name='auc')])
    model.summary()
    return model

def train_model(model, X_train, y_train, X_val, y_val, epochs=100, batch_size=32):
    """ Treina o modelo LSTM com EarlyStopping e ReduceLROnPlateau. """
    print("Iniciando o treinamento do modelo...")
    early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, min_lr=0.0001, verbose=1)
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )
    print("Treinamento concluído.")
    return history

def evaluate_model(model, X_test, y_test, batch_size=32):
    """ Avalia o modelo no conjunto de teste. """
    print("\nAvaliando o modelo no conjunto de teste...")
    results = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0) # verbose=0 para não poluir
    print("-" * 30)
    print(f"Loss no Teste: {results[0]:.4f}")
    print(f"Binary Accuracy no Teste: {results[1]:.4f}")
    if len(results) > 2: print(f"AUC no Teste: {results[2]:.4f}")
    print("-" * 30)
    print("Lembre-se: Métricas refletem o ajuste aos dados históricos, não previsão real.")
    print("-" * 30)
    return results

def predict_next_draw(model, last_sequence, mlb, num_predictions=6):
    """ Faz a previsão para o próximo sorteio. """
    print("\nFazendo a previsão para o PRÓXIMO sorteio...")
    last_sequence_batch = np.expand_dims(last_sequence, axis=0)
    predicted_probabilities = model.predict(last_sequence_batch)[0]
    predicted_indices = np.argsort(predicted_probabilities)[-num_predictions:]
    predicted_numbers = sorted((predicted_indices + 1).tolist()) # Adiciona 1 para obter número real

    print("-" * 30)
    print(f"Previsão dos {num_predictions} números mais prováveis (baseado no histórico):")
    print(predicted_numbers)
    print("-" * 30)
    print("Probabilidades estimadas para os números previstos:")
    for num_idx in predicted_indices:
        print(f"  Número {num_idx+1}: {predicted_probabilities[num_idx]:.4f}")
    print("-" * 30)
    print("AVISO: Esta previsão é um exercício técnico. NÃO HÁ GARANTIA DE ACERTO.")
    print("-" * 30)
    return predicted_numbers, predicted_probabilities

# --- Novas Funções de Visualização ---

def plot_training_history(history):
    """ Plota o histórico de treinamento (loss e AUC). """
    if not history or not history.history:
        print("Histórico de treinamento não disponível para plotar.")
        return

    plt.style.use('seaborn-v0_8-darkgrid') # Estilo do gráfico
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # Gráfico da Loss (Perda)
    if 'loss' in history.history and 'val_loss' in history.history:
        axes[0].plot(history.history['loss'], label='Loss Treinamento')
        axes[0].plot(history.history['val_loss'], label='Loss Validação')
        axes[0].set_title('Histórico da Função de Perda')
        axes[0].set_xlabel('Época')
        axes[0].set_ylabel('Loss (Binary Crossentropy)')
        axes[0].legend()
        axes[0].grid(True)

    # Gráfico da AUC (ou métrica principal)
    metric_key = 'auc' # Prioriza AUC
    val_metric_key = 'val_auc'
    if metric_key not in history.history or val_metric_key not in history.history:
         metric_key = 'binary_accuracy' # Fallback para accuracy
         val_metric_key = 'val_binary_accuracy'

    if metric_key in history.history and val_metric_key in history.history:
        axes[1].plot(history.history[metric_key], label=f'{metric_key.capitalize()} Treinamento')
        axes[1].plot(history.history[val_metric_key], label=f'{metric_key.capitalize()} Validação')
        axes[1].set_title(f'Histórico da Métrica ({metric_key.capitalize()})')
        axes[1].set_xlabel('Época')
        axes[1].set_ylabel(metric_key.capitalize())
        axes[1].legend()
        axes[1].grid(True)
    else:
        # Se nenhuma métrica principal foi encontrada, remove o segundo subplot
        fig.delaxes(axes[1])
        fig.set_size_inches(8, 5) # Ajusta tamanho se for só um gráfico


    plt.suptitle("Desempenho do Modelo Durante o Treinamento")
    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Ajusta layout para o supertítulo


def plot_prediction_distribution(predicted_probabilities, predicted_numbers):
    """ Plota a distribuição de probabilidade da previsão final. """
    if predicted_probabilities is None or len(predicted_probabilities) != NUM_FEATURES:
        print("Probabilidades de previsão inválidas para plotar.")
        return

    plt.figure(figsize=(15, 6))
    numbers = np.arange(1, NUM_FEATURES + 1)
    colors = ['red' if n in predicted_numbers else 'skyblue' for n in numbers]
    bars = plt.bar(numbers, predicted_probabilities, color=colors)

    plt.xlabel("Número da Mega-Sena")
    plt.ylabel("Probabilidade Estimada (Sigmoid Output)")
    plt.title("Distribuição de Probabilidade Estimada para o Próximo Sorteio")
    plt.xticks(np.arange(0, NUM_FEATURES + 1, 5)) # Marcações a cada 5 números
    plt.xlim(0, NUM_FEATURES + 1)
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Adiciona legenda improvisada para as cores
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor='red', edgecolor='red', label='Top 6 Previstos'),
                       Patch(facecolor='skyblue', edgecolor='skyblue', label='Outros Números')]
    plt.legend(handles=legend_elements)


def plot_hits_over_time(model, X_test, y_test, mlb):
    """
    Plota o número de acertos (números sorteados que estavam entre os 6 mais prováveis
    previstos pelo modelo) para cada sorteio no conjunto de teste.
    """
    if X_test is None or y_test is None or X_test.shape[0] == 0:
        print("Dados de teste insuficientes para plotar acertos ao longo do tempo.")
        return

    print("\nCalculando acertos no conjunto de teste (histórico)...")
    y_pred_probs_test = model.predict(X_test)
    hits_per_draw = []

    for i in range(len(y_pred_probs_test)):
        pred_probs = y_pred_probs_test[i]
        actual_encoded = y_test[i]

        # Índices dos 6 números com maior probabilidade prevista
        top6_pred_indices = np.argsort(pred_probs)[-6:]

        # Índices dos números que realmente foram sorteados
        actual_winning_indices = np.where(actual_encoded == 1)[0]

        # Calcula a interseção (quantos números previstos estavam corretos)
        num_hits = len(set(top6_pred_indices) & set(actual_winning_indices))
        hits_per_draw.append(num_hits)

    plt.figure(figsize=(15, 6))
    plt.plot(hits_per_draw, marker='o', linestyle='-', markersize=4, label='Nº de Acertos por Sorteio no Teste')
    plt.xlabel("Índice do Sorteio no Conjunto de Teste (Ordem Cronológica)")
    plt.ylabel("Número de Acertos (entre os Top 6 previstos)")
    plt.title("Número de Acertos do Modelo no Conjunto de Teste Histórico")
    plt.yticks(np.arange(0, 7, 1)) # Eixo Y de 0 a 6 acertos
    plt.grid(True)
    plt.legend()

    # Calcula e exibe estatísticas básicas de acertos
    avg_hits = np.mean(hits_per_draw)
    max_hits = np.max(hits_per_draw)
    print(f"\nEstatísticas de Acertos no Conjunto de Teste ({len(X_test)} sorteios):")
    print(f" - Média de acertos por sorteio: {avg_hits:.3f}")
    print(f" - Máximo de acertos em um sorteio: {max_hits}")
    for i in range(max_hits + 1):
        count = hits_per_draw.count(i)
        print(f" - Sorteios com {i} acerto(s): {count} ({count/len(hits_per_draw)*100:.1f}%)")
    print("Lembre-se: Acertos passados não garantem acertos futuros.")


# --- Nova Função de Exportação ---

def export_predictions_to_excel(model, X, df_original, sequence_length, mlb, filename="mega_sena_predictions.xlsx"):
    """
    Gera previsões para todos os dados sequenciados (X) e exporta para Excel
    junto com os resultados reais correspondentes.
    """
    print(f"\nGerando previsões para exportação para o arquivo: {filename}...")
    if X is None or X.shape[0] == 0:
        print("Não há sequências (X) para gerar previsões para exportação.")
        return

    # 1. Fazer previsões para todas as sequências em X
    all_pred_probs = model.predict(X, batch_size=BATCH_SIZE) # (num_sequences, 60)

    # 2. Preparar DataFrame de probabilidades
    prob_cols = [f'Prob_{i+1}' for i in range(NUM_FEATURES)]
    probs_df = pd.DataFrame(all_pred_probs, columns=prob_cols)

    # 3. Identificar os Top 6 previstos para cada linha
    top_6_preds = []
    for i in range(len(all_pred_probs)):
        top_indices = np.argsort(all_pred_probs[i])[-6:]
        top_numbers = sorted((top_indices + 1).tolist()) # Números reais (1-60)
        top_6_preds.append(top_numbers)

    pred_cols = [f'Pred_{i+1}' for i in range(6)]
    preds_df = pd.DataFrame(top_6_preds, columns=pred_cols)

    # 4. Obter os resultados reais correspondentes às previsões
    # A previsão para X[i] corresponde ao sorteio original em df_original.iloc[i + sequence_length]
    start_index_actual = sequence_length
    end_index_actual = sequence_length + len(X)
    actual_results_df = df_original.iloc[start_index_actual:end_index_actual].reset_index(drop=True)

    # Mantém apenas as colunas originais relevantes (Concurso, Data, Bolas) se existirem
    actual_cols_to_keep = [col for col in ['Concurso', 'Data'] if col in actual_results_df.columns] + \
                          [f'Bola{i+1}' for i in range(6)]
    actual_results_df = actual_results_df[actual_cols_to_keep]

    # 5. Combinar os DataFrames: Reais | Preditos (Top 6) | Probabilidades (Todas)
    final_export_df = pd.concat([actual_results_df, preds_df, probs_df], axis=1)

    # 6. Exportar para Excel
    try:
        final_export_df.to_excel(filename, index=False, engine='openpyxl')
        print(f"Dados exportados com sucesso para '{filename}'.")
        print(f" - O arquivo contém {len(final_export_df)} linhas.")
        print(f" - Cada linha mostra o resultado real e a previsão do modelo para aquele sorteio.")
    except ImportError:
        print("\nAVISO: Para exportar para Excel (.xlsx), a biblioteca 'openpyxl' é necessária.")
        print("Instale com: pip install openpyxl")
    except Exception as e:
        print(f"\nErro ao exportar para Excel: {e}")


# --- Fluxo Principal Atualizado ---
if __name__ == "__main__":
    # 1. Carregar Dados
    df_full = download_and_prepare_data(url=DATA_URL, file_path=DATA_FILE)

    if df_full is not None and not df_full.empty:
        # Extrai apenas as colunas das bolas para pré-processamento
        bola_cols = [f'Bola{i+1}' for i in range(6)]
        df_balls = df_full[bola_cols]

        # 2. Pré-processamento (One-Hot Encoding)
        encoded_data, mlb = preprocess_data(df_balls)

        if encoded_data.shape[0] > SEQUENCE_LENGTH:
            # 3. Criar Sequências
            X, y = create_sequences(encoded_data, SEQUENCE_LENGTH)

            if X.shape[0] > 0:
                # 4. Dividir em Treino, Validação e Teste (CRONOLOGICAMENTE)
                # Teste são os dados mais recentes
                test_split_index = int(len(X) * (1 - TEST_SIZE_RATIO))
                X_temp, X_test = X[:test_split_index], X[test_split_index:]
                y_temp, y_test = y[:test_split_index], y[test_split_index:]

                # Divide o restante em Treino e Validação (aleatório ou cronológico)
                # Usar train_test_split para validação é comum, mas para séries temporais
                # uma divisão cronológica pode ser mais rigorosa. Vamos manter o split aleatório
                # para validação aqui, mas o teste é estritamente cronológico.
                val_size = 0.15 # 15% do que sobrou para validação
                if len(X_temp) > 1: # Precisa de pelo menos 2 amostras para dividir
                    X_train, X_val, y_train, y_val = train_test_split(
                        X_temp, y_temp, test_size=val_size, random_state=42, shuffle=False # shuffle=False para manter ordem local
                    )
                else: # Não há dados suficientes para validação separada
                     X_train, y_train = X_temp, y_temp
                     X_val, y_val = X_test, y_test # Usa teste como validação (não ideal, mas evita erro)
                     print("\nAviso: Poucos dados para conjunto de validação separado. Usando conjunto de teste para validação.")


                print(f"\nDados divididos em:")
                print(f" - Treino:    {len(X_train)} sequências")
                print(f" - Validação: {len(X_val)} sequências")
                print(f" - Teste:     {len(X_test)} sequências (mais recentes)")

                # 5. Construir o Modelo
                model = build_lstm_model(SEQUENCE_LENGTH, NUM_FEATURES, LSTM_UNITS, DROPOUT_RATE)

                # 6. Treinar o Modelo
                history = train_model(model, X_train, y_train, X_val, y_val, EPOCHS, BATCH_SIZE)

                # --- PLOT 1: Histórico de Treinamento ---
                plot_training_history(history)

                # 7. Avaliar o Modelo (no conjunto de teste histórico)
                evaluation_results = evaluate_model(model, X_test, y_test, BATCH_SIZE)

                # --- PLOT 2: Acertos ao Longo do Tempo (no Teste) ---
                plot_hits_over_time(model, X_test, y_test, mlb)

                # 8. Fazer a Previsão para o Próximo Sorteio
                last_sequence_input = encoded_data[-SEQUENCE_LENGTH:]
                if last_sequence_input.shape[0] == SEQUENCE_LENGTH:
                     predicted_numbers, predicted_probabilities = predict_next_draw(model, last_sequence_input, mlb, num_predictions=6)

                     # --- PLOT 3: Distribuição da Previsão Final ---
                     plot_prediction_distribution(predicted_probabilities, predicted_numbers)

                else:
                     print("\nNão foi possível obter a última sequência para previsão final (dados insuficientes).")
                     predicted_probabilities = None # Garante que não tentará plotar

                # 9. Exportar Histórico e Previsões para Excel
                # Usa todas as sequências X (treino+val+teste) para gerar o histórico completo de previsões
                export_predictions_to_excel(model, X, df_full, SEQUENCE_LENGTH, mlb, filename=EXPORT_FILE)

                # Exibe todos os gráficos gerados
                plt.show()

            else:
                print("\nNão foi possível criar sequências a partir dos dados.")
        else:
             print(f"\nDataset muito pequeno ({encoded_data.shape[0]} sorteios) para a sequência de tamanho {SEQUENCE_LENGTH}.")
    else:
        print("\nNão foi possível carregar ou processar os dados da Mega-Sena. Encerrando.")